# Padova Grand Tour - Queries

This notebook will provide ten insightful queries about our data. Get started by starting GraphDB with `docker compose up --force-recreate`.


Sparql wrapper (copy and pasted from the Individual Project's notebooks):

In [126]:

from SPARQLWrapper import SPARQLWrapper, JSON

# Taken from `data/ttlData/ontology.ttl`
prefixString = """
PREFIX owl: <http://www.w3.org/2002/07/owl#> 
PREFIX pgt: <https://padovagrandtour.github.io/entitites#> 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX xml: <http://www.w3.org/XML/1998/namespace> 
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    # Our local endpoint, the repo name is defined in `data/graphdb-repo.ttl` 
    sparql = SPARQLWrapper("http://localhost:7210/repositories/pgt") 
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            array.append(app)
        print(len(array)) # Print the array length
        return array

    except Exception as e :
        print("The operation failed", e)
    

In [127]:
# Improved version that also handles prefixes
mappedPrefixes = {}
for prefix in prefixString.split('\n'):
    if('PREFIX' in prefix):
        prefix = prefix.replace('PREFIX', '', 1).split(':', 1)
        mappedPrefixes[prefix[0].strip()] = prefix[1].strip().replace('<','').replace('>','')

def query(queryString, replacePrefixes=True):
    queryResults = run_query(queryString)
    if(replacePrefixes):
        for queryIndex, queryResultRaw in enumerate(queryResults):
            for subQueryIndex, subQueryResultRaw in enumerate(queryResultRaw):
                queryResult = queryResultRaw[subQueryIndex][1]
                for prefixName, prefixValue in mappedPrefixes.items():
                    queryResult = queryResult.replace(prefixValue, prefixName + ':')

                queryResults[queryIndex][subQueryIndex] = (queryResultRaw[subQueryIndex][0], queryResult)

        return queryResults
    else: return queryResults


Check if everything works:

In [128]:
query("""
select distinct * {
    ?s ?p ?o
}
LIMIT 3
""", replacePrefixes=False)


3


[[('s', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
  ('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
  ('o', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#Property')],
 [('s', 'http://www.w3.org/2000/01/rdf-schema#subPropertyOf'),
  ('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
  ('o', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#Property')],
 [('s', 'http://www.w3.org/2000/01/rdf-schema#subPropertyOf'),
  ('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
  ('o', 'http://www.w3.org/2002/07/owl#TransitiveProperty')]]

## 1 - Get tour elements in order


```
[[('stepIndex', 'rdf:_1'), ('site', 'pgt:SITE1')],
 [('stepIndex', 'rdf:_10'), ('site', 'pgt:SITE10')],
 [('stepIndex', 'rdf:_11'), ('site', 'pgt:SITE11')],
 [('stepIndex', 'rdf:_12'), ('site', 'pgt:SITE12')],
 [('stepIndex', 'rdf:_13'), ('site', 'pgt:SITE13')],
 [('stepIndex', 'rdf:_14'), ('site', 'pgt:SITE14')],
 [('stepIndex', 'rdf:_15'), ('site', 'pgt:SITE15')],
 [('stepIndex', 'rdf:_2'), ('site', 'pgt:SITE2')],
 [('stepIndex', 'rdf:_3'), ('site', 'pgt:SITE3')],
 [('stepIndex', 'rdf:_4'), ('site', 'pgt:SITE4')],
 [('stepIndex', 'rdf:_5'), ('site', 'pgt:SITE5')],
 [('stepIndex', 'rdf:_6'), ('site', 'pgt:SITE6')],
 [('stepIndex', 'rdf:_7'), ('site', 'pgt:SITE7')],
 [('stepIndex', 'rdf:_8'), ('site', 'pgt:SITE8')],
 [('stepIndex', 'rdf:_9'), ('site', 'pgt:SITE9')]]
 ```

 Cause has a lower lexicographical order.

In [129]:
query("""
select ?stepIndex ?stepIndexNumber ?site where {
    BIND (strlen("http://www.w3.org/1999/02/22-rdf-syntax-ns#_") AS ?prefixLength)
	pgt:TOUR0 pgt:steps ?stepNode .
    ?stepNode ?stepIndex ?site
    BIND (xsd:integer(SUBSTR(xsd:string(?stepIndex),?prefixLength + 1)) AS ?stepIndexNumber)

} ORDER BY ?stepIndexNumber

""")


15


[[('stepIndex', 'rdf:_1'), ('stepIndexNumber', '1'), ('site', 'pgt:SITE1')],
 [('stepIndex', 'rdf:_2'), ('stepIndexNumber', '2'), ('site', 'pgt:SITE2')],
 [('stepIndex', 'rdf:_3'), ('stepIndexNumber', '3'), ('site', 'pgt:SITE3')],
 [('stepIndex', 'rdf:_4'), ('stepIndexNumber', '4'), ('site', 'pgt:SITE4')],
 [('stepIndex', 'rdf:_5'), ('stepIndexNumber', '5'), ('site', 'pgt:SITE5')],
 [('stepIndex', 'rdf:_6'), ('stepIndexNumber', '6'), ('site', 'pgt:SITE6')],
 [('stepIndex', 'rdf:_7'), ('stepIndexNumber', '7'), ('site', 'pgt:SITE7')],
 [('stepIndex', 'rdf:_8'), ('stepIndexNumber', '8'), ('site', 'pgt:SITE8')],
 [('stepIndex', 'rdf:_9'), ('stepIndexNumber', '9'), ('site', 'pgt:SITE9')],
 [('stepIndex', 'rdf:_10'), ('stepIndexNumber', '10'), ('site', 'pgt:SITE10')],
 [('stepIndex', 'rdf:_11'), ('stepIndexNumber', '11'), ('site', 'pgt:SITE11')],
 [('stepIndex', 'rdf:_12'), ('stepIndexNumber', '12'), ('site', 'pgt:SITE12')],
 [('stepIndex', 'rdf:_13'), ('stepIndexNumber', '13'), ('site', 'pg

## 2 -

## 3 -

## 4 -

## 5 -

## 6 -

## 7 -

## 8 -

## 9 -

## 10 -